<a href="https://colab.research.google.com/github/MalihaT111/ai-recruitment/blob/main/AI_Assisted_Recruitment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats

In [ ]:
resume_df = pd.read_csv('Resume.csv')
resume_df.shape
resume_df.head()
resume_df.isnull().sum()

FileNotFoundError: [Errno 2] No such file or directory: 'Resume.csv'

In [ ]:
job_posts_df = pd.read_csv('data job posts.csv')
job_posts_df.shape
job_posts_df.head()
job_posts_df.isnull().sum()

In [ ]:
#Columns with missing values, turning them into a list
condition = job_posts_df.isnull().sum() != 0
job_posts_df.isnull().sum()[condition].index
columnlist = list(job_posts_df.isnull().sum()[condition].index)
columnlist

In [ ]:
job_posts_df[columnlist].dtypes
#Everything is object so meaning there's no numerical value

In [ ]:
for column in columnlist:
    print(job_posts_df[column].unique())



In [ ]:
#outliers

## Addressing Null values in job_posts_df
Since there are many columns in job_posts_df with null values, we can use reasoning to drop some of the rows or columns. Some columns can be cut if they have too many null values or if they are not really relevant to the problem.  For example, the column "AnnouncementCode" has 17793 null values and the unique non-null values are a random string of letters. It was most likely used to identify the job posting on its original website. As well, "Opening Date" and "Deadline" may not have many null values, but these values might not be very useful to determine whether or not a candidate would be a good fit for a job.

In [ ]:
columns_to_drop = ['AnnouncementCode', 'Term', 'Eligibility', 'Audience', 'StartDate', 'Duration', 'OpeningDate', 'Deadline', 'Notes', 'Attach']
job_posts_df = job_posts_df.drop(columns=columns_to_drop)
job_posts_df.isnull().sum()

There are still null values, but the columns are too contextually important to the ML problem. We can drop the examples that have null values in these columns since they most likely do not have the information we need to train the model accurately. After we drop these rows, our job_posts_df dataset no longer has any null values.

In [ ]:
columns_to_check = ['Title', 'JobDescription', 'JobRequirment', 'Company', 'Location', 'RequiredQual', 'Salary', 'AboutC']

job_posts_df = job_posts_df.dropna(subset=columns_to_check)

job_posts_df.shape

In [ ]:
job_posts_df.isnull().sum()

In [ ]:
#Removing duplicates
job_posts_df = job_posts_df.drop_duplicates()
print(job_posts_df.duplicated().sum())
resume_df = resume_df.drop_duplicates()
print(resume_df.duplicated().sum())

In [ ]:
#Check which column has HTML tags
import re
def has_html(text):
    if isinstance(text, str):
        return bool(re.search(r'<.*?>', text))
    return False

In [ ]:
columns_with_html = [col for col in job_posts_df.columns if job_posts_df[col].apply(has_html).any()]
print(columns_with_html)

columns_with_html = [col for col in resume_df.columns if resume_df[col].apply(has_html).any()]
print(columns_with_html)
#So no column in job_post has any HTML tags we need to remove. Only resume :/

In [ ]:
import re
def clean_html(text):
  return re.sub('<[^<]+?>', '', text)

resume_df['Resume_html'] = resume_df['Resume_str'].apply(clean_html)

In [ ]:
resume_df[['Resume_html', 'Resume_str']].head()
#They're the same thing so i'm dropping the HTML column :D
resume_df = resume_df.drop(columns=['Resume_html'], inplace= True)